In [2]:
# Generates an equivalent figure to Fig. 1 in the paper but fitting cubic splines to
# estimate ensemble mean values rather than cubic polynomials.

import climepi  # noqa
import holoviews as hv
from climepi import climdata, epimod

import opts
from figure_export import export_main_figure

# Get options
options = opts.get_opts()
year_range = options["year_range"]
location = options["location_default"]
epi_model_name = options["epi_model_name_default"]
ensemble_stats_kwargs = {
    **options["ensemble_stats_kwargs"],
    "internal_variability_method": "splinefit",
}
plot_opts_clim = options["plot_opts_clim"]
plot_opts_epi = options["plot_opts_epi"]
data_base_dir = options["data_base_dir"]

colors = hv.Cycle().values

2025-05-01 16:02:40,162 [INFO]: logger.py(log:11) >> ====== WebDriver manager ======
2025-05-01 16:02:40,162 [INFO]: logger.py(log:11) >> ====== WebDriver manager ======
2025-05-01 16:02:40,650 [INFO]: logger.py(log:11) >> Get LATEST chromedriver version for google-chrome
2025-05-01 16:02:40,650 [INFO]: logger.py(log:11) >> Get LATEST chromedriver version for google-chrome
2025-05-01 16:02:40,708 [INFO]: logger.py(log:11) >> Get LATEST chromedriver version for google-chrome
2025-05-01 16:02:40,708 [INFO]: logger.py(log:11) >> Get LATEST chromedriver version for google-chrome
2025-05-01 16:02:40,740 [INFO]: logger.py(log:11) >> There is no [mac64] chromedriver "135.0.7049.114" for browser google-chrome "135.0.7049" in cache
2025-05-01 16:02:40,740 [INFO]: logger.py(log:11) >> There is no [mac64] chromedriver "135.0.7049.114" for browser google-chrome "135.0.7049" in cache
2025-05-01 16:02:40,741 [INFO]: logger.py(log:11) >> Get LATEST chromedriver version for google-chrome
2025-05-01 16

In [3]:
# Get climate data
ds_clim = (
    climdata.get_example_dataset("isimip_cities_daily", base_dir=data_base_dir)
    .sel(location=location)
    .sel(time=slice(str(year_range[0]), str(year_range[1])))
)
# Yearly average temperature data
ds_temp_yearly = ds_clim.climepi.yearly_average("temperature")
ds_temp_yearly["time"] = ds_temp_yearly["time.year"]  # avoids plotting issues

In [4]:
# Get and run epidemiological model
epi_model = epimod.get_example_model(epi_model_name)
ds_suitability = ds_clim.climepi.run_epi_model(epi_model)
ds_days_suitable = ds_suitability.climepi.yearly_portion_suitable()
ds_days_suitable["time"] = ds_days_suitable["time.year"]  # avoids plotting issues

In [5]:
# Example scenarios and models
scenario_ex = "ssp370"
model_ex = "ipsl-cm6a-lr"
scenarios_compare = ["ssp126", "ssp370", "ssp585"]
scenarios_compare_plot_names = ["SSP1-2.6", "SSP3-7.0", "SSP5-8.5"]
models_compare = ["canesm5", "ipsl-cm6a-lr", "miroc6"]
models_compare_plot_names = ["CanESM5", "IPSL-CM6A-LR", "MIROC6"]

In [6]:
# Internal variability
ds_plot_internal = ds_temp_yearly.sel(scenario=scenario_ex, model=model_ex)
p_internal_plume = ds_plot_internal.climepi.plot_uncertainty_interval_decomposition(
    kwargs_baseline={"label": "Ensemble mean"}, **ensemble_stats_kwargs
)
p_internal_ex = ds_plot_internal.climepi.plot_time_series(
    label="Ensemble member", line_color="gray"
)
p_internal = (p_internal_plume * p_internal_ex).opts(**plot_opts_clim)
p_internal

:Overlay
   .Area.Internal_variability :Area   [time]   (lower,upper)
   .Curve.Ensemble_mean       :Curve   [time]   (temperature)
   .Curve.Ensemble_member     :Curve   [time]   (Temperature)

In [7]:
# Model variability
ds_plot_model = (
    ds_temp_yearly.sel(scenario=scenario_ex, model=models_compare)
    .climepi.ensemble_stats(**ensemble_stats_kwargs)
    .sel(stat="mean", drop=True)
    .assign_coords(model=models_compare_plot_names)
    .rename(model="Climate model")
)
p_model = ds_plot_model.climepi.plot_time_series(
    by="Climate model", line_color=colors[1], line_dash=["dashed", "solid", "dotted"]
).opts(**{**plot_opts_clim})
p_model

:NdOverlay   [Climate model]
   :Curve   [time]   (Temperature)

In [8]:
# Scenario variability
ds_plot_scenario = (
    ds_temp_yearly.sel(scenario=scenarios_compare)
    .climepi.ensemble_stats(**ensemble_stats_kwargs)
    .sel(stat="mean", drop=True)
    .mean(dim="model")
    .assign_coords(scenario=scenarios_compare_plot_names)
    .rename(scenario="Scenario")
)
p_scenario = ds_plot_scenario.climepi.plot_time_series(
    by="Scenario", line_color=colors[2], line_dash=["dashed", "solid", "dotted"]
).opts(**plot_opts_clim)
p_scenario

:NdOverlay   [Scenario]
   :Curve   [time]   (temperature)

In [9]:
# Climate variability
p_clim_plume = ds_temp_yearly.climepi.plot_uncertainty_interval_decomposition(
    **ensemble_stats_kwargs
).opts(**plot_opts_clim)
p_clim_plume

:Overlay
   .Area.Scenario_uncertainty :Area   [time]   (lower,upper)
   .Area.I                    :Area   [time]   (lower,upper)
   .Area.Model_uncertainty    :Area   [time]   (lower,upper)
   .Area.II                   :Area   [time]   (lower,upper)
   .Area.Internal_variability :Area   [time]   (lower,upper)
   .Curve.Mean                :Curve   [time]   (temperature)

In [10]:
# Epi variability
p_epi_plume = ds_days_suitable.climepi.plot_uncertainty_interval_decomposition(
    **ensemble_stats_kwargs
).opts(**plot_opts_epi)
p_epi_plume

:Overlay
   .Area.Scenario_uncertainty :Area   [time]   (lower,upper)
   .Area.I                    :Area   [time]   (lower,upper)
   .Area.Model_uncertainty    :Area   [time]   (lower,upper)
   .Area.II                   :Area   [time]   (lower,upper)
   .Area.Internal_variability :Area   [time]   (lower,upper)
   .Curve.Mean                :Curve   [time]   (portion_suitable)

In [11]:
export_main_figure(
    [p_internal, p_model, p_scenario, p_clim_plume, p_epi_plume],
    file_name="sensitivity_splines.svg",
)

In [14]:
# Example proportions of uncertainty due to internal variability in a single year

year_example = 2080


def _get_icv_uncertainty_proportion(ds, time=None, data_var=None):
    da_decomp = ds.climepi.uncertainty_interval_decomposition(
        data_var, **ensemble_stats_kwargs
    )[data_var].sel(time=time)
    internal_width = (
        da_decomp.sel(level="internal_upper") - da_decomp.sel(level="internal_lower")
    ).values
    total_width = (
        da_decomp.sel(level="scenario_upper") - da_decomp.sel(level="scenario_lower")
    ).values
    return internal_width / total_width


prop_temp = _get_icv_uncertainty_proportion(
    ds_temp_yearly, time=year_example, data_var="temperature"
)
print(
    f"Proportion of uncertainty in temperature in {year_example} "
    f"due to internal variability: {prop_temp * 100:.2f}%"
)
prop_epi = _get_icv_uncertainty_proportion(
    ds_days_suitable, time=year_example, data_var="portion_suitable"
)
print(
    f"Proportion of uncertainty in months suitable in {year_example} "
    f"due to internal variability: {prop_epi * 100:.2f}%"
)

Proportion of uncertainty in temperature in 2080 due to internal variability: 37.35%
Proportion of uncertainty in months suitable in 2080 due to internal variability: 47.07%
